# Mini-lab 4 - Mapping TAZ travel times

In this lab we will play with some data from the Metropolitan Transportation Commission on travel time from one Traffic Analysis Zone (TAZ) to another. We will use a mapping tool called folium to create a graphical representation of travel times throughout the area.


In [ ]:
from datascience import *
import numpy as np


# this is a bug fix! I'll explain why it's needed during the mini-lab. For now just run the code:
def _map_copy(self):
    my_map =  Map(features=self._features, width=self._width,
               height=self._height, **self._attrs)
    my_map._folium_map  =self._folium_map
    return my_map

Map.copy = _map_copy

## The datasets
### MTC travel skims
The Metropolitan Transportation Commission (MTC) is the regional transportation planning organization for the Bay Area. They host a database with average travel time, cost, and distance from each traffic analysis zone (TAZ) to all other TAZs in the Bay Area. The files have data for driving alone, carpooling, walking to transit, driving to transit, walking, and biking. 

We have pre-processed the data from the morning commute to include only TAZs around San Francisco, Oakland and Berkeley. The files with inter-TAZ travel time, travel cost, and travel distance are saved with the following filepaths 'data/sf_oak_TimeSkims_AM.csv', 'data/sf_oak_CostSkims_AM.csv' and 'data/sf_oak_DistanceSkims_AM.csv'

More info on the dataset can be found here - http://analytics.mtc.ca.gov/foswiki/Main/SimpleSkims. 
The descriptions of the columns in the data set are shown below:

|column|description|
|---|---|
|origin|Origin transportation analysis zone|
|destination|Destination transportation analysis zone|
|drive alone|Door-to-door time for the drive alone travel mode (i.e. single occupant private automobile)|
|shared ride (2 people)|Door-to-door time for the shared ride 2 travel mode (i.e. double occupant private automobile)|
|shared ride (3 people)|Door-to-door time for the shared ride 3+ travel mode (i.e. three-or-more occupants traveling in a private vehicle)|
|walk|Door-to-door time for walking|
|bike|Door-to-door time for bicycling|
|walk-transit-walk|Door-to-door time for walk to transit to walk paths|
|drive-transit-walk|Door-to-door time for drive to transit to walk paths|
| walk-transit-drive|Door-to-door time for walk to transit to drive paths (returning home on a park-and-ride tour)|


(The raw data with all Bay Area TAZs can be found at https://mtcdrive.app.box.com/2015-03-116)

### Bay Area TAZ geometry data
GeoJSON is a format used for encoding a variety of geographic data structures. We have saved a GeoJSON file with the Traffic Analysis Zone (TAZ) polygons for the TAZs in the San Francisco, Oakland, and Berkeley region. We will use a mapping package called folium to map the TAZs.

### Read the data
First we read in the sf_oak_TimeSkims_AM.csv using the Table.read_table() method

In [ ]:
# Read the timeskims datafile
travel_time_data = Table.read_table('sf_oak_traveltimes_bymode.csv')
travel_time_data

**Question:** what is the drive alone travel time from origin TAZ 10 (in downtown SF) to destination TAZ 
1019 (the TAZ at UC Berkeley) according to this dataset?

Hint: use a where() statement to find the answer.

In [ ]:
# Your code and anwer here:


# Part 2: Use the datascience map tool to map the TAZs

As mentioned above, geojson is a standard format for encoding geographic data. 'SF_Oak_TAZs.geojson' is a geojson file that contains the TAZ geometry for TAZs in the San Francisco and Oakland area. We can use the datascience Map tool to map the polygons in the geojson file.

In [ ]:
base_taz_map = Map.read_geojson('SF_Oak_TAZs.geojson')
base_taz_map

In [ ]:
print (base_taz_map._folium_map)

# Create a table of the map features
The geojson contains a few hundred unique polygon features, one for each TAZ. In the cell below we can use the Map objects .map_features property to get a list of the polygon features. Then we can use the Table.from_records() method to convert this to a table.

The corresponding table has 5 total columns, 3 corresponding to the properties in the geojson feature:

|Column| Description|
|---|---|
|County_FIP|An id for the county a TAZ is contained in|
|TAZ1454| A numeric TAZ id.'TAZ1454' refers to the fact that in total there are 1454 TAZs in the Bay Area, and they are assigned ids from 1 to 1454. Only about 300 are shown in the map above.|
|UrbanTAZac|TODO fill this in.|
|feature | info on the polygon itself (it is displayed as a map in the table) |
| id | a newly assigned Table id, with values from 0 to the length of the table - 1.|


In [ ]:
taz_table = Table.from_records(base_taz_map.features)

# Note - DO NOT use the Table.show() method here - it will likely make your notebook crash!
taz_table

# Using maps to convey information
## Color TAZs based on TAZ id
First, we might want to know the general location of the various TAZ ids. We can use the Map.color() method to color the map based on the value in a table. To do this we:

1. Create a 2-column table, column 1 should contain the feature ids and column 2 should contain the information used to color the table.
2. Use the map.color(values, ids=(), key_on='feature.id', palette='YlOrBr') method to color the map. The inputs to this method are:
 * values = the 2-column table feature described in #1, 
 * key_on = to the name of the map property to be matched to the ids in the table, in this case 'feature.properties.TAZ1454'. 
 * Optional 
    * color palette: 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'
    * fill_opacity: 1 for totally opaque, 0 for totally transparent
    * threshold_scale (where the color breaks are).

In [ ]:
# 1. Create a 2-column table with columns "ids" and "values". Since we want to color based on 
# the value of the id, in this case both the value and id column are set to the TAZ1454 values. 

tocolor_table = Table().with_columns(['ids', taz_table.column('TAZ1454'), 
                                      'values', taz_table.column('TAZ1454')])
tocolor_table

In [ ]:
# 2. Use the map.color() function to color the maps based on the values defined in our tocolor_table.
base_taz_map.color(values=tocolor_table,key_on='feature.properties.TAZ1454', 
                   threshold_scale=[100,200,400,1000,1200],palette='YlOrRd',fill_opacity=1.)

**Question:** From the map above, what city would you guess the TAZ with TAZ1454=27
is located in?


In [ ]:
# Your answer here:

# Conveying more interesting information

## Color the map based on the 'bike' travel time from one origin TAZ to all other TAZs.
In the next few cells we will create a **very cool** data visualization tool to visualize the travel time from an origin TAZ to all destination TAZs.

1. Use a where statement to get only the rows of the travel_time_data where the origin == origin_taz_id. Save the result as a new table called tt_from_origin
2. Use the Table.select() method to select only the destination column and the column that contains the travel time by 'bike' from the tt_from_origin table. Save this result as tt_from_origin.
3. Use taz_map.color() to map the results

In [ ]:
origin_taz_id = 10
mode = 'bike'

threshold_scale = [0,5,10,15,20,25] #sets where the color transitions are

# Your task: 1. Use a where statement to get only the rows of the travel_time_data where the 
# origin == origin_taz_id. Save the result as a new table called tt_from_origin
tt_from_orig = #FILL THIS IN




In [ ]:
# Your task: 2. Use the Table.select() method to select only the destination column 
# and the column that contains the travel time by 'bike' from the tt_from_origin
# table. Save this result as tt_from_origin_mode.
tt_from_orig_mode = #FILL THIS IN

The results should look something like:

|destination|bike|
|---|---|
|1|7.85|
|2|7.55|
|3|7.2|
|4|	6.6|
|5|	5.7|
|6|	5.7|
|7|	4.9|
|8|	3.9|
|9|	2.45|
|10|1.05|

Run the following to see the result:

In [ ]:
taz_map = base_taz_map.color(tt_from_orig_mode, key_on='feature.properties.TAZ1454', palette='YlOrRd',
                             threshold_scale=threshold_scale, fill_opacity=.8)
taz_map

# Use the overlay method to color the origin TAZ blue

### Example of how Map.overlay() works:

In [ ]:
# finds the taz with TAZ1454 id = origin_taz_id by using the .where() method on the taz_table
features = taz_table.where(taz_table['TAZ1454']==origin_taz_id)['feature']

# Use the overlay method to color the feature blue
taz_map.overlay(features, 'Blue', .9)

# Color unreachable TAZs grey
For the datasets provided, MTC follows a convention of setting travel time = '-999' in the data table if they have determined that a mode is not feasible to get from an origin TAZ to a destination TAZ. For example, a bike trip from San Francisco to Oakland is not possible because bikes are not allowed accross the Bay Bridge. Similarly some walk distances are determined to be too far to make a trip on foot.
Color all TAZs with a travel time of -999 grey. Hint - use a procedure similar to the procedure used to color the origin blue.

In [ ]:
# Tazs that are unreachable by a certain mode have a travel time of -999
# Your task: use the Table.where() method on the tt_from_orig_mode table 
# to get rows where travel time =-999. 

unreachable_tazs = #FILL THIS IN


In [ ]:
# Run the following to get a list taz_ids of all the unreachable tazs:
unreachable_taz_ids = unreachable_tazs['destination']

# Now we need to select only the features from the taz_table that have TAZ1454 in the
# unreachable list. Use the .where function to select these features. You may want to 
# use the np.in1d() method that returns True if an element is in a list. Once we have a 
# list of features, use the Map.overlay method to add them to our taz_map

# np.in1d example:
# np.in1d([3,5,2], [1,2,3])


features = taz_table.where(# FILL THIS IN)


In [ ]:
taz_map.overlay(features, 'Grey', .8)

# Put it all in one method

Create a function that produces a travel time map. You have all of the components above, just fill them in to the function below.

In [ ]:
def create_tt_map(data, base_taz_map, origin_taz_id=1, mode='drive alone', color_palette='YlOrRd', 
                  threshold_scale=[5,10,15,20,25]):

    #1. create the map:
    tt_from_orig = # FILL THIS IN
    tt_from_orig_mode = # FILL THIS IN

    taz_map = base_taz_map.color(tt_from_orig_mode, key_on='feature.properties.TAZ1454', palette=color_palette,
                            threshold_scale=threshold_scale, fill_opacity=0.8)
    
    #2. create the taz_table from the map
    taz_table = Table.from_records(taz_map.features)
    
    
    #3. color the unreachable tazs grey:
    unreachable_tazs = #FILL THIS IN
    features = #FILL THIS IN
    taz_map.overlay(#fill this in)
    
    #4. color the origin blue
    features = #FILL THIS IN
    taz_map.overlay(#fill this in)
    
    return taz_map

# Test your method
By running the following code. 

In [ ]:
create_tt_map(travel_time_data, taz_map, origin_taz_id = 917, mode='bike')

# Explore travel times for other modes of transportation:
Use the method you created to plot the walk-transit-walk travel times from TAZ 5. Try a different color palette and modify the threshold_scale so the color transitions occur at appropriate levels.

# The relationship between spatial proximity and travel time
For each of the following modes: drive, walk to transit, drive to transit, bike, and walk, do you think a straightline distance from one TAZ to another is a good indicator of travel time? Explain why or why not.



In [ ]:
# Your answer here:



# Bonus - adding Circle marker labels to the map
The following code returns a list of lats, and a list of lons for the centroid of each taz and a list of TAZ labels. Create a Circle marker table as we did in minilab 3. Overlay the circle marker on our TAZ map. 

In [ ]:
from shapely import geometry
import json

features = taz_table['feature']
lats, lons, labels = [], [], []
for i in range(len(taz_table.column('id'))):
    ct = geometry.shape(features[i].geojson(0)['geometry']).centroid
    lats.append(ct.xy[1][0])
    lons.append(ct.xy[0][0])
    labels.append('TAZ '+str(taz_table['TAZ1454'][i]))

# Your task - create a circle marker table (with columns latitude, longitude, popup, 'color' and 'radius')
# as we did in minilab 3


In [ ]:
# Your task: use the overlay method to add these markers to our TAZ map
